In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functions import *

In [3]:
def resample_trajectory(trajectory: np.ndarray, T: int) -> np.ndarray:
    """
    Linearly resample a discrete trajectory to length T.
    """
    t_old = np.linspace(0.0, 1.0, len(trajectory))
    t_new = np.linspace(0.0, 1.0, T)

    return np.vstack([
        np.interp(t_new, t_old, trajectory[:, d])
        for d in range(trajectory.shape[1])
    ]).T

## Relative Velocity Function (RVF)

In [3]:
def relative_velocity_function(
    alpha_ref: np.ndarray,
    alpha: np.ndarray,
    T: int,
    align_fn,
) -> np.ndarray:
    """
    Algorithm 1: Relative Velocity Function (RVF)

    Parameters
    ----------
    alpha_ref : ndarray (N, D)
        Reference trajectory α_T(t)
    alpha : ndarray (M, D)
        Target trajectory α(t)
    T : int
        Uniform resampling length
    align_fn : callable
        Function γ* = Align(alpha_ref, alpha)

    Returns
    -------
    rvf : ndarray (T-1,)
        Discrete relative velocity function
    """
    # Step 1: Resample
    alpha_ref_r = resample_trajectory(alpha_ref, T)
    alpha_r = resample_trajectory(alpha, T)

    # Step 2: Optimal temporal alignment
    gamma_star = align_fn(alpha_ref_r, alpha_r)  # shape (T,)

    # Step 3: Discrete derivative of γ*
    rvf = np.diff(gamma_star)

    return rvf


## Spatial Asymmetry

In [ ]:
def spatial_asymmetry(
    alpha: np.ndarray,
    T: int,
    reflection_matrix: np.ndarray,
    distance_fn,
) -> np.ndarray:
    """
    Algorithm 2: Spatial Asymmetry (SA)

    Parameters
    ----------
    alpha : ndarray (N, D)
        Discrete trajectory α(t)
    T : int
        Uniform resampling length
    reflection_matrix : ndarray (D, D)
        Left-right reflection matrix R
    distance_fn : callable
        d_C(x, y) distance in shape space

    Returns
    -------
    sS : ndarray (T,)
        Spatial asymmetry function
    """
    # Step 1: Resample
    alpha_r = resample_trajectory(alpha, T)

    sS = np.zeros(T)

    # Step 2–4: Reflect and compute distance
    for t in range(T):
        alpha_reflected = reflection_matrix @ alpha_r[t]
        sS[t] = distance_fn(alpha_r[t], alpha_reflected)

    return sS


## Temporal Asymmetry

In [4]:
def temporal_asymmetry(
    alpha: np.ndarray,
    beta: np.ndarray,
    T: int,
    align_fn,
    distance_fn,
) -> np.ndarray:
    """
    Algorithm 3: Temporal Asymmetry (TA)

    Parameters
    ----------
    alpha : ndarray (N, D)
        Trajectory α(t)
    beta : ndarray (M, D)
        Trajectory β(t)
    T : int
        Uniform resampling length
    align_fn : callable
        Function γ* = Align(alpha, beta)
    distance_fn : callable
        d_S([x], [y]) distance in shape space

    Returns
    -------
    tS : ndarray (T,)
        Temporal asymmetry function
    """
    # Step 1: Resample
    alpha_r = resample_trajectory(alpha, T)
    beta_r = resample_trajectory(beta, T)

    # Step 2: Optimal alignment
    gamma_star = align_fn(alpha_r, beta_r)  # shape (T,)

    tS = np.zeros(T)

    # Step 3–4: Time-reversed comparison
    for t in range(T):
        idx = int(np.round(gamma_star[T - 1 - t]))
        idx = np.clip(idx, 0, T - 1)
        tS[t] = distance_fn(alpha_r[t], beta_r[idx])

    return tS
